<a href="https://colab.research.google.com/github/Rossel/DataQuest_Courses/blob/master/028__Conditional_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COURSE 3/6: STORYTELLING THROUGH DATA VISUALIZATION

# MISSION 4: Conditional_plots

In this mission, we will explore how to ....

## 1. Introduction to Seaborn

So far, we've mostly worked with plots that are quick to analyze and make sense of. Line charts, scatter plots, and bar plots allow us to convey a few nuggets of insights to the reader. We've also explored how we can combine those plots in interesting ways to convey deeper insights and continue to extend the storytelling power of data visualization. In this mission, we'll explore how to quickly create multiple plots that are subsetted using one or more conditions.

We'll be working with the seaborn visualization library, which is built on top of matplotlib. [Seaborn](http://seaborn.pydata.org/) has good support for more complex plots, attractive default styles, and integrates well with the pandas library. Here are some examples of some complex plots that can be created using seaborn:

![Seaborn](https://s3.amazonaws.com/dq-content/seaborn_gallery.png)




## 2. Introduction to the Data Set

We'll be working with a data set of the passengers of the Titanic. The [Titanic shipwreck](https://en.wikipedia.org/wiki/Sinking_of_the_RMS_Titanic) is the most famous shipwreck in history and led to the creation of better safety regulations for ships. One substantial safety issue was that there were not enough lifeboats for every passenger on board, which meant that some passengers were prioritized over others to use the lifeboats.

The data set was compiled by [Kaggle](https://www.kaggle.com/c/titanic/data) for their introductory data science competition, called **Titanic: Machine Learning from Disaster**. The goal of the competition is to build machine learning models that can predict if a passenger survives from their attributes. 

The data for the passengers is contained in two files:

- `train.csv`: Contains data on 712 passengers
- `test.csv`: Contains data on 418 passengers

Each row in both data sets represents a passenger on the Titanic, and some information about them. We'll be working with the `train.csv` file, because the `Survived` column, which describes if a given passenger survived the crash, is preserved in the file. The column was removed in `test.csv`, to encourage competitors to practice making predictions using the data. Here are descriptions for each of the columns in `train.csv`:

- `PassengerId` -- A numerical id assigned to each passenger.
- `Survived` -- Whether the passenger survived (`1`), or didn't (`0`).
- `Pclass` -- The class the passenger was in.
- `Name` -- the name of the passenger.
- `Sex` -- The gender of the passenger -- male or female.
- `Age` -- The age of the passenger. Fractional.
- `SibSp` -- The number of siblings and spouses the passenger had on board.
- `Parch` -- The number of parents and children the passenger had on board.
- `Ticket` -- The ticket number of the passenger.
- `Fare` -- How much the passenger paid for the ticket.
- `Cabin` -- Which cabin the passenger was in.
- `Embarked` -- Where the passenger boarded the Titanic.

### **Importing the data**

Let's start by importing the libraries we need and reading the dataset into pandas using Google Colab.


In [1]:
# Import functions from Google modules into Colaboratory
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# Insert file id from Google Drive shareable link:
# https://drive.google.com/file/d/1vQzWwsOM8Besa36uhMdm3JkiIXeLtJpL/view?usp=sharing
id = "1vQzWwsOM8Besa36uhMdm3JkiIXeLtJpL"

In [3]:
# Download the dataset
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')

In [4]:
# Import pandas library
import pandas as pd
import numpy as np

The dataset could not be read using "UTF-8" or "Windows-1252" encoding, so we used "Latin-1".

In [5]:
 # Read encoded csv
 titanic = pd.read_csv("train.csv", encoding='Latin-1')

### **Exploring the data**
Let's render the first few and last few values of this pandas object, by running the `train` variable in a separate cell.

In [6]:
# Render the first few and last rows of the autos dataframe
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


###**Remove missing values**



Let's remove columns like `Name` and `Ticket` that we don't have a way to visualize. Keep only the following columns:
- `Survived`
- `Pclass`
- `Sex`
- `Age`
- `SibSp`
- `Parch`
- `Fare`
- `Embarked`

In addition, we need to remove any rows containing missing values, as **seaborn will throw errors when we try to plot missing values**. We will use the [`DataFrame.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to remove rows containing missing values.

Recall that we can use the `DataFrame.isnull()` method to identify missing values, which returns a boolean dataframe. We can then use the DataFrame.sum() method to give us a count of the True values for each column:

In [7]:
print(titanic.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
